In [65]:
import io
import torch
import torch.onnx
import onnx
import torch.nn.functional as F
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import torch.nn as nn
# from utils.dataset import BasicDataset

In [66]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
pthfile = 'ckpt.test.pth'
onnxpath = 'test.onnx'
print(torch.__version__)
print(onnx.__version__)

1.4.0
1.10.1


In [71]:
class Flatten(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x.view(x.shape[0], -1)

actor = nn.Sequential(  # (8, G, G)
            nn.Conv2d(8, 8, 3, padding=1),  # (8, G, G)
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 4, 3, padding=1),  # (4, G, G)
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.Conv2d(4, 4, 5, padding=2),  # (4, G, G)
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.Conv2d(4, 2, 5, padding=2),  # (2, G, G)
            nn.BatchNorm2d(2),
            nn.ReLU(),
            nn.Conv2d(2, 1, 5, padding=2),  # (1, G, G)
            #nn.BatchNorm2d(1),
            Flatten(),  # (G*G, )
            #nn.Sigmoid(),  # added for non-negative
        )

In [72]:
# map_location = 'gpu' if torch.cuda.is_available() else 'cpu'
actor.load_state_dict(torch.load(pthfile, map_location='cpu'),strict=False)

_IncompatibleKeys(missing_keys=['0.weight', '0.bias', '1.weight', '1.bias', '1.running_mean', '1.running_var', '3.weight', '3.bias', '4.weight', '4.bias', '4.running_mean', '4.running_var', '6.weight', '6.bias', '7.weight', '7.bias', '7.running_mean', '7.running_var', '9.weight', '9.bias', '10.weight', '10.bias', '10.running_mean', '10.running_var', '12.weight', '12.bias'], unexpected_keys=['module.0.weight', 'module.0.bias', 'module.1.weight', 'module.1.bias', 'module.1.running_mean', 'module.1.running_var', 'module.1.num_batches_tracked', 'module.3.weight', 'module.3.bias', 'module.4.weight', 'module.4.bias', 'module.4.running_mean', 'module.4.running_var', 'module.4.num_batches_tracked', 'module.6.weight', 'module.6.bias', 'module.7.weight', 'module.7.bias', 'module.7.running_mean', 'module.7.running_var', 'module.7.num_batches_tracked', 'module.9.weight', 'module.9.bias', 'module.10.weight', 'module.10.bias', 'module.10.running_mean', 'module.10.running_var', 'module.10.num_batches

In [70]:
def test():
    G=240
    model = actor
    batch_size = 1
    input_shape = (8, G, G)
    
    # Initialize model with the pretrained weights
    map_location = 'gpu' if torch.cuda.is_available() else 'cpu'
    loaded_model = torch.load(pthfile, map_location=map_location)
    model.load_state_dict(loaded_model, strict=False)
    
    # set the model to inference mode
    model.eval()
    
    # data type nchw
    x = torch.rand(batch_size, *input_shape)
    input_names = ['input']
    output_names = ['output']
    # # Export the model
    torch.onnx.export(model,               # model being run
                    x,                         # model input (or a tuple for multiple inputs)
                    onnxpath,   # where to save the model (can be a file or file-like object)
                    export_params=True,        # store the trained parameter weights inside the model file
                    opset_version=10,          # the ONNX version to export the model to
                    do_constant_folding=True,  # whether to execute constant folding for optimization
                    input_names = ['input'],   # the model's input names
                    output_names = ['output'], # the model's output names
                    dynamic_axes={'input' : {0 : 'batch_size'},    # variable lenght axes
                                  'output' : {0 : 'batch_size'}}
                     )

In [62]:
test()

In [44]:
"""
注意，模型的输入大小和测试的输入数据一致；
注意，在导出模型之前，请先调用torch_model.eval()或torch_model.train(False)，以将模型转换为推理模式，这一点很重要。 这是必需的，因为像dropout或batchnorm这样的运算符在推断和训练模式下的行为会有所不同。
注意，除非指定为动态轴，否则输入尺寸将在导出的 ONNX 图中固定为所有输入尺寸。
在此示例中，我们使用输入batch_size=1导出模型，但随后在torch.onnx.export()的dynamic_axes参数中将第一维指定为动态。 因此，导出的模型将接受大小为[batch_size,3, 640, 959]的输入，其中batch_size可以是可变的。
"""

'1.10.1'